The Schreier-Sims algorithm computes the order of a group $G$ by recursively calculating the orders of a chain of subgroups $G = G_0 \geq G_1 \geq \dots \geq G_r = \{e\}$, where $G_i$ is the subgroup of G that fixes the points $\{1, 2, \dots, i\}$.

The recursive procedure works as follows:
*   The function takes a set of generators for a group Gᵢ.
*   Apply the stripping algorithm to this set to prevent exploding growth of the number of generators.
*   If the set of generators is empty thenthe current group is trivial.
*   The algorithm finds the first point $\alpha$ that is not fixed by all generators. This point is guaranteed to have a non-trivial orbit and will be $i+1$ in the stabiliser chain.
*   It then computes the orbit of α under the action of $G_i$ and records its size $|\mathop{Orb}(\alpha)|$.
*   Find a generating set for the stabilier of $\alpha$, which is our next group in the chain $G_{i+1}$.
*   The function calls itself with the newly found generators for $G_{i+1}$ and returns the order $|G_{i+1}|$.
*   The order of the current group $G_i$ is then calculated using the orbit-stabiliser theorem and is returned up the recursion chain.
*   The final result is the product of the sizes of all the orbits found at each step of the chain.

In [59]:
import collections
import copy

class Permutation:
    '''
    A class to represent a permutation of the set {1, 2, ..., n}.
    '''
    def __init__(self, p_list):
        self._p = list(p_list)
        self.size = len(p_list)

    def __len__(self):
        return self.size

    def __repr__(self):
        return f"Perm({self._p})"

    def __eq__(self, other):
        if not isinstance(other, Permutation):
            return NotImplemented
        return self._p == other._p

    def __getitem__(self, i):
        if not 1 <= i <= self.size:
            raise IndexError(f"Index {i} is out of bounds for size {self.size}.")
        return self._p[i - 1]

    def __mul__(self, other):
        if not isinstance(other, Permutation) or self.size != other.size:
            raise ValueError("Permutations must be of the same size.")
        new_p_list = [self[other[i]] for i in range(1, self.size + 1)]
        return Permutation(new_p_list)

    def inverse(self):
        inv_p_list = [0] * self.size
        for i in range(1, self.size + 1):
            j = self[i]
            inv_p_list[j - 1] = i
        return Permutation(inv_p_list)

    def is_identity(self):
        for i in range(self.size):
            if self._p[i] != i + 1: return False
        return True

def stripping_algorithm_procedure(initial_generators):
    if not initial_generators: return []
    n = initial_generators[0].size
    gen_array = [[None for _ in range(n)] for _ in range(n)]
    for original_gen in initial_generators:
        p = copy.deepcopy(original_gen)
        for i in range(1, n + 1):
            j = p[i]
            if i == j: continue
            existing_gen = gen_array[i - 1][j - 1]
            if existing_gen is None:
                gen_array[i - 1][j - 1] = p
                break
            else:
                p = existing_gen.inverse() * p
    reduced_generators = [gen for row in gen_array for gen in row if gen is not None]
    return reduced_generators

def compute_orbit_with_witnesses(generators, alpha):
    n = generators[0].size
    identity = Permutation(list(range(1, n + 1)))
    queue = collections.deque([alpha])
    witnesses = {alpha: identity}
    while queue:
        beta = queue.popleft()
        witness_beta = witnesses[beta]
        for gen in generators:
            gamma = gen[beta]
            if gamma not in witnesses:
                witnesses[gamma] = gen * witness_beta
                queue.append(gamma)
    return witnesses

def compute_stabilizer_generators(initial_generators, alpha):
    witness_map = compute_orbit_with_witnesses(initial_generators, alpha)
    T = list(witness_map.values())
    def phi(g): return witness_map[g[alpha]]
    schreier_generators = []
    for y in initial_generators:
        for t in T:
            g = y * t
            s = phi(g).inverse() * g
            if not s.is_identity():
                schreier_generators.append(s)
    if not schreier_generators: return []
    return stripping_algorithm_procedure(schreier_generators)

In [60]:
def compute_group_order_recursive(generators, depth=0):
    '''
    Recursively computes the order of a group using the Schreier-Sims algorithm.
    '''
    indent = "  " * depth
    print(f"{indent}--- Recursion level {depth} ---")

    # Log and strip the incoming generators
    print(f"{indent}Generators received: {len(generators)}")
    stripped_gens = stripping_algorithm_procedure(generators)
    print(f"{indent}Generators after stripping: {len(stripped_gens)}")

    # Base case: If no generators are left, we have the trivial group of order 1.
    if not stripped_gens:
        print(f"{indent}Base case reached. This is the trivial group.")
        print(f"{indent}Subgroup order: 1")
        return 1

    n = stripped_gens[0].size

    # Find the first element alpha not fixed by all generators
    alpha = -1
    for i in range(1, n + 1):
        is_fixed = all(g[i] == i for g in stripped_gens)
        if not is_fixed:
            alpha = i
            break

    # Compute the orbit of alpha
    orbit_map = compute_orbit_with_witnesses(stripped_gens, alpha)
    orbit_size = len(orbit_map)
    print(f"{indent}Found nontrivial orbit for alpha = {alpha} of size {orbit_size}")

    # Compute generators for the stabilizer of alpha
    stabilizer_gens = compute_stabilizer_generators(stripped_gens, alpha)

    # Recurse on the stabilizer subgroup
    stabilizer_order = compute_group_order_recursive(stabilizer_gens, depth + 1)

    # Use the Orbit-Stabilizer theorem to find the order of the current group
    group_order = orbit_size * stabilizer_order
    print(f"{indent}Subgroup order = {orbit_size} (orbit size) * {stabilizer_order} (stabilizer order) = {group_order}")

    return group_order

Forgetting to use the stripping algorithm at every stage would be very detrimental for the performance of the algorithm, especially for larger groups. The number of Schreier generators produced passed to the next recursive step for a stabilizer $G_\alpha$ is $km$, where $k$ is the number of generators for $G$ and $m$ is the size of the orbit of $\alpha$. The step after that would receive $kmm'$ generators, where $m'$ is the size of the next orbit.

The number of generators grows exponentially. Since the complexity of all our procedures is at least linear in the number of generators $k$, the runtime of each recursive step would become impossibly long. Furthermore, the memory required to store these redundant permutations would quickly be exhausted. The stripping algorithm ensures that the number of generators for any subgroup of $S_n$ is kept below $n(n-1)/2$. This keeps k small and manageable, making the entire computation feasible.

In [61]:
def compute_group_order(initial_generators, group_name="G"):
    if not initial_generators:
        print("Received empty set of generators. Order is 1.")
        return 1
    print(f"\n=== Computing Order of Group {group_name} ===")
    order = compute_group_order_recursive(initial_generators)
    print(f"=== Order of {group_name}: {order} ===")
    return

s3_gens = [Permutation([2, 1, 3]), Permutation([2, 3, 1])]
compute_group_order(s3_gens, group_name="S3")

d8_gens = [Permutation([2, 3, 4, 1]), Permutation([4, 3, 2, 1])]
compute_group_order(d8_gens, group_name="D8")

s4_gens = [Permutation([2, 1, 3, 4]), Permutation([2, 3, 4, 1])]
compute_group_order(s4_gens, group_name="S4")


=== Computing Order of Group S3 ===
--- Recursion level 0 ---
Generators received: 2
Generators after stripping: 2
Found nontrivial orbit for alpha = 1 of size 3
  --- Recursion level 1 ---
  Generators received: 1
  Generators after stripping: 1
  Found nontrivial orbit for alpha = 2 of size 2
    --- Recursion level 2 ---
    Generators received: 0
    Generators after stripping: 0
    Base case reached. This is the trivial group.
    Subgroup order: 1
  Subgroup order = 2 (orbit size) * 1 (stabilizer order) = 2
Subgroup order = 3 (orbit size) * 2 (stabilizer order) = 6
=== Order of S3: 6 ===

=== Computing Order of Group D8 ===
--- Recursion level 0 ---
Generators received: 2
Generators after stripping: 2
Found nontrivial orbit for alpha = 1 of size 4
  --- Recursion level 1 ---
  Generators received: 1
  Generators after stripping: 1
  Found nontrivial orbit for alpha = 2 of size 2
    --- Recursion level 2 ---
    Generators received: 0
    Generators after stripping: 0
    Base 

For the group $S_n$ we consider the probability $\Pr(n)$ that a pair of elements $g, h$ picked uniformly at random generates $S_n$,
\begin{equation}
    \Pr(n) =  \frac{|\{(g, h) \in S_n \times S_n : ⟨g, h⟩ = S_n\}|}{|Sn|^2}.
\end{equation}
We know that $\Pr(n) > 0$ as $S_n$ can be generated by just two elements, for instance, a transposition $g = (1\;2)$ and a long cycle $h = (1\;\dots\;n)$.

We claim that there is a $k < 1$ independent of $n$ such that $\Pr(n) \leq k$. To find an upper bound $k < 1$, we need to identify a large, proper subgroup $H \leq S_n$ calculate the probability that two randomly chosen elements both fall into $H$. If $g, h \in H$, then $\langle g, h\rangle \leq H$ cannot be $S_n$.

Consider the alternating group $A_n$ which is the subgroup of all even permutations in $S_n$. The size of $S_n$ is $n!$ and the size of $A_n$ is $n!/2$. The probability that a single random permutation $g \in S_n$ is an even permutation is
\begin{equation}
    \Pr(g \in A_n) = \frac{|A_n|}{|S_n|} = \frac{(n!/2)}{n!} = \frac{1}{2},
\end{equation}
The choices of g and h are independent, therefore the probability that both $g$ and $h$ are even perutations is $1/4$. This means that at least $1/4$ of all possible pairs fail to generate $S_n$. Therefore, the probability $\Pr(n)$ that a pair does generate $S_n$ must be less than or equal to $3/4$. This of course is a very crude estimate of the true value.

To generate a permutation of $\{1, \dots, n\}$ uniformly at random, we can use the Fisher-Yates shuffle algorithm.
*   Start with an ordered list, $[1, 2, \dots, n]$.
*   Iterate backwards from $i = n$ down to $2$.
*   In each step, generate a random integer $j$ such that $1 \leq j \leq i$.
*   Swap the elements at positions $i-1$ and $j-1$ in the list
*   The resulting list is a uniformly random permutation.

In [62]:
import random
import math
import collections
import copy

class Permutation:
    '''
    A class to represent a permutation of the set {1, 2, ..., n}.
    '''
    def __init__(self, p_list):
        self._p = list(p_list)
        self.size = len(p_list)
    def __len__(self): return self.size
    def __repr__(self): return f"Permutation({self._p})"
    def __eq__(self, other): return isinstance(other, Permutation) and self._p == other._p
    def __getitem__(self, i): return self._p[i - 1]
    def __mul__(self, other): return Permutation([self[other[i]] for i in range(1, self.size + 1)])
    def inverse(self):
        inv = [0] * self.size
        for i in range(1, self.size + 1): inv[self[i] - 1] = i
        return Permutation(inv)
    def is_identity(self): return all(self._p[i] == i + 1 for i in range(self.size))

def stripping_algorithm_procedure(gens):
    if not gens: return []
    n = gens[0].size
    arr = [[None for _ in range(n)] for _ in range(n)]
    for g in gens:
        p = copy.deepcopy(g)
        for i in range(1, n + 1):
            j = p[i]
            if i == j: continue
            if arr[i - 1][j - 1] is None:
                arr[i - 1][j - 1] = p
                break
            else:
                p = arr[i - 1][j - 1].inverse() * p
    return [g for row in arr for g in row if g is not None]

def compute_orbit_with_witnesses(gens, alpha):
    n = gens[0].size
    id = Permutation(list(range(1, n + 1)))
    q, w = collections.deque([alpha]), {alpha: id}
    while q:
        b = q.popleft()
        for g in gens:
            c = g[b]
            if c not in w:
                w[c] = g * w[b]
                q.append(c)
    return w

def compute_stabilizer_generators(gens, alpha):
    w_map = compute_orbit_with_witnesses(gens, alpha)
    T = list(w_map.values())
    phi = lambda g: w_map[g[alpha]]
    sch_gens = [(phi(y*t).inverse() * (y*t)) for y in gens for t in T]
    return stripping_algorithm_procedure([g for g in sch_gens if not g.is_identity()])

def compute_group_order_recursive(gens):
    stripped = stripping_algorithm_procedure(gens)
    if not stripped: return 1
    n = stripped[0].size
    alpha = next((i for i in range(1, n + 1) if any(g[i] != i for g in stripped)), -1)
    orbit_size = len(compute_orbit_with_witnesses(stripped, alpha))
    stab_gens = compute_stabilizer_generators(stripped, alpha)
    return orbit_size * compute_group_order_recursive(stab_gens)

In [63]:
def generate_random_permutation(n):
    '''
    Generates a random permutation of size n using Fisher-Yates shuffle.
    '''
    p = list(range(1, n + 1))
    for i in range(n, 1, -1):
        j = random.randint(1, i) - 1
        p[i-1], p[j] = p[j], p[i-1]
    return Permutation(p)

def estimate_pn(n, num_trials=100):
    '''
    Estimates the probability P_n that two random permutations generate S_n.
    '''
    order_of_sn = math.factorial(n)
    generation_count = 0

    for i in range(num_trials):
        g = generate_random_permutation(n)
        h = generate_random_permutation(n)
        generated_group_order = compute_group_order_recursive([g, h])
        if generated_group_order == order_of_sn:
            generation_count += 1

    estimate = generation_count / num_trials
    print(f"Estimated Pr({n}) ≈ {estimate:.2f}")
    return estimate

# Estimate P_n for a few moderate values of n
for n in range(1, 10):
    estimate_pn(n)

Estimated Pr(1) ≈ 1.00
Estimated Pr(2) ≈ 0.71
Estimated Pr(3) ≈ 0.54
Estimated Pr(4) ≈ 0.36
Estimated Pr(5) ≈ 0.44
Estimated Pr(6) ≈ 0.40
Estimated Pr(7) ≈ 0.64
Estimated Pr(8) ≈ 0.54
Estimated Pr(9) ≈ 0.65


These estimates suggest that the probability $\Pr(n)$ is not only greater than our theoretical bound of $3/4$, but that it also appears to be increasing as $n$ increases. This makes sense, as our bound only measures the failure to generate the alternating group and ignores any other subgroups. Dixon's Theorem states that the probability $\Pr(n) \to 1$ as $n \to \infty$. In other words, for a large $n$, two randomly chosen permutations will almost certainly generate the entire symmetric group $S_n$.